In [ ]:
import cv2 as cv
import numpy as np
import os

dataSet = 'C:\\Users\\alanj\\Desktop\\IA\\Proyectos\\Emociones\\DataSet'
faces = os.listdir(dataSet)
print(faces)

labels = []
facesData = []
label = 0

for face in faces:
    facePath = os.path.join(dataSet, face)
    for faceName in os.listdir(facePath):
        labels.append(label)
        facesData.append(cv.imread(os.path.join(facePath, faceName), 0))
    label += 1

print(np.count_nonzero(np.array(labels) == 0))

faceRecognizer = cv.face.LBPHFaceRecognizer_create()

faceRecognizer.train(facesData, np.array(labels))

faceRecognizer.save('C:\\Users\\alanj\\Desktop\\IA\\Proyectos\\Emociones\\EIGENFACE.xml')

In [ ]:
import cv2 as cv
import os

# Cargar el reconocedor y el clasificador de rostros solo una vez
faceRecognizer = cv.face.LBPHFaceRecognizer_create()
faceRecognizer.read('C:\\Users\\alanj\\Desktop\\IA\\Proyectos\\Emociones\\EIGENFACE.xml')

rostro = cv.CascadeClassifier('C:\\Users\\alanj\\Desktop\\IA\\Proyectos\\Emociones\\haarcascade_frontalface_alt.xml')

# Define la ruta del dataset y obtén las etiquetas
dataSet = 'C:\\Users\\alanj\\Desktop\\IA\\Proyectos\\Emociones\\DataSet'
faces = os.listdir(dataSet)
print(faces)

# Iniciar la captura de video
cap = cv.VideoCapture(0)

# Opcional: reducir la resolución de captura para mejorar el rendimiento
cap.set(cv.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, 480)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cpGray = gray.copy()
    
    # Detectar rostros en la imagen
    rostros = rostro.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in rostros:
        frame2 = cpGray[y:y+h, x:x+w]
        frame2 = cv.resize(frame2, (100, 100), interpolation=cv.INTER_CUBIC)
        
        result = faceRecognizer.predict(frame2)
        cv.putText(frame, '{}'.format(result), (x, y-20), 1, 1.3, (0, 0, 0), 1, cv.LINE_AA)
        
        if result[1] < 400:
            cv.putText(frame, '{}'.format(faces[result[0]]), (x, y-25), 2, 1.1, (0, 255, 0), 1, cv.LINE_AA)
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        else:
            cv.putText(frame, 'Desconocido', (x, y-20), 2, 0.8, (0, 0, 255), 1, cv.LINE_AA)
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
    
    cv.imshow('frame', frame)
    
    # Salir del bucle si se presiona la tecla 'ESC'
    if cv.waitKey(1) == 27:
        break

cap.release()
cv.destroyAllWindows()


